In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions.feedback import KMeansSegmentation, VectorSegmenter, HyperplaneSegmentation
from functions.funcs import get_kxk_square, get_kxk_square_values, assess_square, signal_results, row_agg, kmeans_step, plane_step, initialize_segmenters
from functions.classifiers3x3 import c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13
from functions.classifierskxk import bars

# Create vector of zeros length 364, to be modified when learning starts
kmeans_amps = np.zeros(364)
plane_amps = np.zeros(364)

test = pd.read_csv('digit-recognizer/test.csv')
train = pd.read_csv('digit-recognizer/train.csv')
labels = train.iloc[:, 0] 
data = train.iloc[:, 1:]  

In [5]:
# Set variables for the image, kxk square size, and starting index
k = 5
w, h = 28, 28
p = 98

# Select an image from the dataset
image = 1937

# Get the column indeces for the kxk square, then get the specific values for the selected image
pix = get_kxk_square(p = p, k = k)
values_list = get_kxk_square_values(r = image, k = k, pix = pix, data = data)

# Print as a little matrix
print(values_list)


[[  0   0   0   0   0]
 [  0   0   0   0   0]
 [144 173 254 148  65]
 [196 253 253 253 237]
 [  6  44  44 202 253]]


In [6]:
# Assess interesting parts of the square
## avg_value, dark_cols, dark_rows, light_cols, light_rows, is_interesting
signals = assess_square(values_list = values_list)
print(signals)

classifications = bars(signals=signals, values_list=values_list)
print(classifications)

(101.0, False, True, False, True, True)
['c12']


In [8]:
k = len(values_list)
nleft = np.floor(k / 2).astype(int)
nright = np.ceil(k / 2).astype(int)

avg = signals[0]
dark_cols = signals[1]
dark_rows = signals[2]
light_cols = signals[3]
light_rows = signals[4]
is_interesting = signals[5]

dark_threshold = avg + 40
light_threshold = avg - 40
dark_mask = values_list >= dark_threshold
light_mask = values_list <= light_threshold

print(light_mask)

# any(np.all(r) for r in [light_mask[:nleft+n,:].tolist() for n in range(nright-1)])
print([np.all(r) for r in [light_mask[:nleft+n+1,:].tolist() for n in range(nright-1)]])

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [False False False  True  True]
 [False False False False  True]
 [False False False False  True]]
[False, False]


In [6]:
# 3x3 Assessment

############## Empty/Full ##############
print(f"Empty sensor returns: {c0(signals=signals)}")
print(f"Full sensor returns: {c1(signals=signals)}")
############## Columns ##############
print(f"Left dark vbar sensor returns: {c2(values_list=values_list, signals=signals)}")
print(f"Right light vbar sensor returns: {c3(values_list=values_list, signals=signals)}")
print(f"Middle light vbar sensor returns: {c4(values_list=values_list, signals=signals)}")
print(f"Middle dark vbar sensor returns: {c5(values_list=values_list, signals=signals)}")
print(f"Left light vbar sensor returns: {c6(values_list=values_list, signals=signals)}")
print(f"Right dark vbar sensor returns: {c7(values_list=values_list, signals=signals)}")
############## Rows ##############
print(f"Top dark hbar sensor returns: {c8(values_list=values_list, signals=signals)}")
print(f"Bottom light hbar sensor returns: {c9(values_list=values_list, signals=signals)}")
print(f"Middle light hbar sensor returns: {c10(values_list=values_list, signals=signals)}")
print(f"Middle dark hbar sensor returns: {c11(values_list=values_list, signals=signals)}")
print(f"Top light hbar sensor returns: {c12(values_list=values_list, signals=signals)}")
print(f"Bottom dark hbar sensor returns: {c13(values_list=values_list, signals=signals)}")

Empty sensor returns: False
Full sensor returns: False
Left dark vbar sensor returns: False
Right light vbar sensor returns: False
Middle light vbar sensor returns: False
Middle dark vbar sensor returns: False
Left light vbar sensor returns: False
Right dark vbar sensor returns: False
Top dark hbar sensor returns: False
Bottom light hbar sensor returns: False
Middle light hbar sensor returns: False
Middle dark hbar sensor returns: False
Top light hbar sensor returns: False
Bottom dark hbar sensor returns: True


In [7]:
# 5x5 assessment
classifications = bars(signals=signals, values_list=values_list)
print(classifications) 

['c12']


In [5]:
n = 784
image = 1937

binary_matrix = signal_results(values_list=values_list, data=data, image=image)
# print(binary_matrix)

In [6]:
result_vector = row_agg(binary_matrix=binary_matrix)
# print("Result vector:", result_vector)
# print(len(result_vector))

In [7]:
kmeans_output = result_vector + kmeans_amps
plane_output = result_vector + plane_amps
# print(output)
# print(len(output))

In [8]:
kmeans_segmenter, plane_segmenter = initialize_segmenters(num_sections=11, len=364)